In [56]:
import requests
import datetime
import base64
from urllib.parse import urlencode
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [2]:
client_id = '86241dc85057485b9d020a85673e5c1d'
client_secret = '67e6e25d4d5c43e3abb36860e8fc1355'

In [98]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        #print(query_params)
        return self.base_search(query_params)

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [72]:
tracks = pd.read_csv('../data/unique_songs.csv')

In [73]:
tracks.head()

,Unnamed: 0,song,artist,primary_artist,featured_artist_1,featured_artist_2
0,0,Easy On Me,Adele,Adele,NaN,NaN
1,1,Stay,The Kid LAROI & Justin Bieber,The Kid LAROI,Justin Bieber,NaN
2,2,Industry Baby,Lil Nas X & Jack Harlow,Lil Nas X,Jack Harlow,NaN
3,3,Fancy Like,Walker Hayes,Walker Hayes,NaN,NaN
4,4,Bad Habits,Ed Sheeran,Ed Sheeran,NaN,NaN


In [74]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29681 entries, 0 to 29680
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         29681 non-null  int64 
 1   song               29681 non-null  object
 2   artist             29681 non-null  object
 3   primary_artist     29681 non-null  object
 4   featured_artist_1  4367 non-null   object
 5   featured_artist_2  592 non-null    object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


In [75]:
tracks_spotify = tracks[["song","primary_artist"]]

In [76]:
tracks_spotify.head()

,song,primary_artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI
2,Industry Baby,Lil Nas X
3,Fancy Like,Walker Hayes
4,Bad Habits,Ed Sheeran


In [77]:
tracks_spotify["is_collected"] = False

D:\Users\Alfaisal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
tracks_spotify

,song,primary_artist,is_collected
0,Easy On Me,Adele,False
1,Stay,The Kid LAROI,False
2,Industry Baby,Lil Nas X,False
3,Fancy Like,Walker Hayes,False
4,Bad Habits,Ed Sheeran,False
...,...,...,...
29676,Stay,The Ames Brothers,False
29677,Over And Over,Thurston Harris,False
29678,Little Serenade,The Ames Brothers,False
29679,I'll Get By (As Long As I Have You),Billy Williams,False


In [100]:
def get_info(i):
    sp_song = tracks_spotify.loc[i,"song"]
    sp_artist = tracks_spotify.loc[i,"primary_artist"]
    spot_song = spotify.search({"track": sp_song , "artist":sp_artist }, search_type='Track')
    tracks_spotify.loc[i,"match_song"] = spot_song['tracks']['items'][0]["name"]
    tracks_spotify.loc[i,"duration_ms"] = spot_song['tracks']['items'][0]["duration_ms"]
    tracks_spotify.loc[i,"spotify_popularity"] = spot_song['tracks']['items'][0]["popularity"]
    tracks_spotify.loc[i,"song_id"] = spot_song['tracks']['items'][0]["id"]
    tracks_spotify.loc[i,"song_uri"] = spot_song['tracks']["items"][0]["uri"]
    
    #artist info
    tracks_spotify.loc[i,"match_artist"] =spot_song['tracks']['items'][0]['artists'][0]['name']
    tracks_spotify.loc[i,"artist_id"] = spot_song['tracks']["items"][0]["artists"][0]['id']
    tracks_spotify.loc[i,"artist_id"] = spot_song['tracks']["items"][0]["artists"][0]['uri']
    #album info
    tracks_spotify.loc[i,"release_date"] = spot_song['tracks']["items"][0]["album"]['release_date']
    tracks_spotify.loc[i,"Album_id"] = spot_song['tracks']["items"][0]["album"]['id']
    tracks_spotify.loc[i,"album_type"] = spot_song['tracks']["items"][0]["album"]['album_type']
    tracks_spotify.loc[i,"album_name"] = spot_song['tracks']["items"][0]["album"]['name']
    tracks_spotify.loc[i,"album_uri"] = spot_song['tracks']["items"][0]["album"]['uri']
    
    tracks_spotify.loc[i,"is_collected"] = True

In [104]:
for i in tqdm(range(2999,3100)):
    try:
        if tracks_spotify.loc[i,"is_collected"] == False:
            if spotify.perform_auth() == True:
                get_info(i)
    except:
        continue

  0%|          | 0/101 [00:00<?, ?it/s]

q=track%3ABlow+Your+Mind+%28Mwah%29+artist%3ADua+Lipa&type=track


  2%|▏         | 2/101 [00:00<00:46,  2.11it/s]

q=track%3ADifferent+For+Girls+artist%3ADierks+Bentley&type=track


  3%|▎         | 3/101 [00:01<00:57,  1.69it/s]

...

 98%|█████████▊| 99/101 [01:19<00:01,  1.28it/s]

q=track%3ACut+It+artist%3AO.T.+Genasis&type=track


 99%|█████████▉| 100/101 [01:19<00:00,  1.29it/s]

q=track%3AOui+artist%3AJeremih&type=track


100%|██████████| 101/101 [01:20<00:00,  1.25it/s]


In [85]:
##tracks_spotify.to_csv("../data/tracks_spotify.csv")

In [113]:
#tracks_spotify[(tracks_spotify["match_artist"] != tracks_spotify["primary_artist"]) & (tracks_spotify["is_collected"] == True)]

In [88]:
loaded = pd.read_csv("../data/tracks_spotify.csv")

In [90]:
loaded[loaded["is_collected"]==True]

,Unnamed: 0,song,primary_artist,is_collected,match_song,duration_ms,spotify_popularity,song_id,song_uri,match_artist,artist_id,release_date,Album_id,album_type,album_name,album_uri
0,0,Easy On Me,Adele,True,Easy On Me,224694.0,95.0,0gplL1WMoJ6iYaPgMCL0gX,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,Adele,spotify:artist:4dpARuHxo51G3z768sgnrY,2021-10-14,224jZ4sUX7OhAuMwaxp86S,single,Easy On Me,spotify:album:224jZ4sUX7OhAuMwaxp86S
1,1,Stay,The Kid LAROI,True,STAY (with Justin Bieber),141805.0,96.0,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,The Kid LAROI,spotify:artist:2tIP7SsRs7vjIcLrU85W8J,2021-07-09,4QLAtpLNUsHEYrcHXmMIZZ,single,STAY (with Justin Bieber),spotify:album:4QLAtpLNUsHEYrcHXmMIZZ
2,2,Industry Baby,Lil Nas X,True,INDUSTRY BABY (feat. Jack Harlow),212000.0,95.0,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,Lil Nas X,spotify:artist:7jVv8c5Fj3E9VhNjxT4snq,2021-07-23,622NFw5Yk0OReMJ2XWcXUh,single,INDUSTRY BABY (feat. Jack Harlow),spotify:album:622NFw5Yk0OReMJ2XWcXUh
3,3,Fancy Like,Walker Hayes,True,Fancy Like,161853.0,81.0,3ZozFqJJlDBNIm4xqxn2ZD,spotify:track:3ZozFqJJlDBNIm4xqxn2ZD,Walker Hayes,spotify:artist:7sKxqpSqbIzphAKAhrqvlf,2021-06-04,1Lcuv3HMgEG36SkibQIspv,single,Country Stuff,spotify:album:1Lcuv3HMgEG36SkibQIspv
4,4,Bad Habits,Ed Sheeran,True,Bad Habits,230746.0,95.0,3rmo8F54jFF8OgYsqTxm5d,spotify:track:3rmo8F54jFF8OgYsqTxm5d,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,2021-10-29,32iAEBstCjauDhyKpGjTuq,album,=,spotify:album:32iAEBstCjauDhyKpGjTuq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,CRZY,Kehlani,True,CRZY,191720.0,62.0,6Dwtha2FtZFoMEBh5GR2sq,spotify:track:6Dwtha2FtZFoMEBh5GR2sq,Kehlani,spotify:artist:0cGUm45nv7Z6M6qdXYQGTX,2017-01-27,4B4in9QlrlYWSHlYSRebdC,album,SweetSexySavage (Deluxe),spotify:album:4B4in9QlrlYWSHlYSRebdC
2996,2996,Lockjaw,French Montana,True,Lockjaw (feat. Kodak Black),223146.0,63.0,7iaw359G2XT14uTfV9feip,spotify:track:7iaw359G2XT14uTfV9feip,French Montana,spotify:artist:6vXTefBL93Dj5IqAWq6OTv,2016-05-27,7sSW3tQBtQrO7f6uO4Bcxq,single,Lockjaw (feat. Kodak Black),spotify:album:7sSW3tQBtQrO7f6uO4Bcxq
2997,2997,You & Me,Marc E. Bassy,True,You & Me,218493.0,68.0,7yCqehT9dKACTFy7YugP0J,spotify:track:7yCqehT9dKACTFy7YugP0J,Marc E. Bassy,spotify:artist:3tQx1LPXbsYjE9VwN1Peaa,2017-10-13,4OmqJXPhD5FOqwdg5tBsnu,album,Gossip Columns,spotify:album:4OmqJXPhD5FOqwdg5tBsnu
2998,2998,For Free,DJ Khaled,True,For Free (feat. Drake),183695.0,58.0,5TqBAOIBe0bQA0spNdcG3k,spotify:track:5TqBAOIBe0bQA0spNdcG3k,DJ Khaled,spotify:artist:0QHgL1lAIqAw0HtD7YldmP,2016-08-05,5qHyKDjwycRj1fJyPLt2HQ,album,Major Key,spotify:album:5qHyKDjwycRj1fJyPLt2HQ


In [114]:
tracks_spotify.to_csv("../data/tracks_spotify_1.csv")